In [16]:
import mediapipe as mp
import cv2
import numpy as np

In [17]:
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [18]:
def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180.0:
        angle=360-angle
    return angle
    

In [19]:
cap=cv2.VideoCapture(0)
counter=0
stage=None
cap=cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while True:
        res,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        results=pose.process(image)
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        try:
            landmarks=results.pose_landmarks.landmark
            shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #calculate angle
            angle=calculate_angle(shoulder,elbow,wrist)
            #visualize angle
            cv2.putText(image,str(angle),tuple(np.multiply(elbow,[640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
            #curl counter logic 
            if  angle > 160:
              stage="down"
            elif angle < 30 and stage=="down":
              stage="up"
              counter=counter+1
        except:
            pass
        #render curl counter
        cv2.rectangle(image,(0,0),(100,100),(0,0,0),-1)
        cv2.putText(image,"CURL", (22,22),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
        cv2.putText(image,str(counter),(10,80),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                           mp_drawing.DrawingSpec(color=(233,22,10),thickness=2,circle_radius=2),mp_drawing.DrawingSpec(color=(22,100,222),thickness=2,circle_radius=2))
        cv2.imshow("Exercise",image)
        if cv2.waitKey(10) & 0xFF==ord("q"):
          break
    cap.release()
    cv2.destroyAllWindows()
